In [19]:
import os
from django.contrib.gis.gdal import DataSource

chlmex_201810 = os.path.abspath('/Users/russmo/Code/wdpa/CEA_MEX_CHL_edits_20191010.gpkg')
chlmex_201810_gdb = os.path.abspath('/Users/russmo/Code/wdpa/CEA_CHL_MEX_edits_20181010.gdb')

In [20]:
ds = DataSource(chlmex_201810_gdb)
print(ds)
print(len(ds))

/Users/russmo/Code/wdpa/CEA_CHL_MEX_edits_20181010.gdb (OpenFileGDB)
1


In [22]:
chlmex = ds[0]
for lyr in ds:
    print (lyr, lyr.geom_type, len(lyr), lyr.srs,)
    f_types = [fld.__name__ for fld in lyr.field_types]
    # print (list(zip(lyr.fields, f_types, lyr.field_widths)))

CEA_CHL_MEX_edits_20181010_poly MultiPolygon 174 GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]


In [33]:
from mpa.models import Mpa
src = chlmex
print('Importing', len(src), 'records from MPA edit table')

Importing 174 records from Source table


In [339]:
allfields = src.fields.copy()
removefields = [
    'mpa_id',
    'summary',
    'fishing_info',
    'geom_Length',
    'geom_Area',
    'status_year_nonull',
    'Shape_Length',
    'Shape_Area'
]
fields = [f for f in allfields if f not in removefields]

In [352]:
for feat in src:
    mid = feat['mpa_id'].value
    print(mid, feat, feat['mpa_id'])
    m, created = Mpa.objects.get_or_create(name=feat['name'].value, designation=feat['designation'].value)
    for f in fields:
        setattr(m, f, feat[f].value)
    m.geom = feat.geom.geos
    m.save()
    print(feat.fid, mid, flush=True)
    # print(feat.fid, mid, end='\r', flush=True)

68808648 Feature FID 1 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808648
Carto Error for mpa_id 68808648: Access denied
1 68808648
68808647 Feature FID 2 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808647


GEOS_NOTICE: Ring Self-intersection at or near point -74.555738684999938 -47.542473715999961

GEOS_NOTICE: Ring Self-intersection at or near point -74.555738684999938 -47.542473715999961



Carto Error for mpa_id 68808647: Access denied
2 68808647
68808646 Feature FID 3 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808646
Carto Error for mpa_id 68808646: Access denied
3 68808646
68808645 Feature FID 4 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808645
Carto Error for mpa_id 68808645: Access denied
4 68808645
68808644 Feature FID 5 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808644
Carto Error for mpa_id 68808644: Access denied
5 68808644
68808643 Feature FID 6 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808643
Carto Error for mpa_id 68808643: Access denied
6 68808643
68808642 Feature FID 7 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808642
Carto Error for mpa_id 68808642: Access denied
7 68808642
68808641 Feature FID 8 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808641
Carto Error for mpa_id 68808641: Access denied
8 68808641
68808640 Feature FID 9 in Layer<CEA_CHL_MEX_edits_20181010_poly> 68808640
Carto Error for mpa_id 68808640: Access denied
9 68808640
68808639 Feature F

MultipleObjectsReturned: get() returned more than one Mpa -- it returned 2!

In [163]:
import urllib.request, json 
for feat in src:
    mid = feat['mpa_id'].value
    if len(feat['summary'].value) >= 255:
        print(feat['name'].value, "http://www.mpatlas.org/mpa/sites/%s/json/" % (mid))
        try:
            with urllib.request.urlopen("http://www.mpatlas.org/mpa/sites/%s/json/" % (mid)) as url:
                data = json.loads(url.read().decode())
                print(mid, len(data['summary']))
                m = Mpa.objects.filter(mpa_id=mid)
                print(len(m.first().summary))
                # m.update(summary=data['summary'])
        except(urllib.request.HTTPError):
            print(mid, 'Not on MPAtlas production server')
            m = Mpa.objects.filter(mpa_id=mid)
            print(len(m.first().summary))

Tierra Blanca http://www.mpatlas.org/mpa/sites/68808633/json/
68808633 Not on MPAtlas production server
3685
Lobería Selkirk http://www.mpatlas.org/mpa/sites/68808632/json/
68808632 Not on MPAtlas production server
3685
El Arenal http://www.mpatlas.org/mpa/sites/68808631/json/
68808631 Not on MPAtlas production server
3685
Montes submarinos Crusoe y Selkirk http://www.mpatlas.org/mpa/sites/68808630/json/
68808630 Not on MPAtlas production server
3685
Mar de Juan Fernández http://www.mpatlas.org/mpa/sites/68808392/json/
68808392 3685
3685
Nazca-Desventuradas http://www.mpatlas.org/mpa/sites/9175/json/
9175 5767
5767
Pacifico Mexicano Profundo - Core Zones http://www.mpatlas.org/mpa/sites/68808479/json/
68808479 6306
6306
Juan Fernández http://www.mpatlas.org/mpa/sites/68808467/json/
68808467 5024
5024
Mar de Cabo de Hornos (Diego Ramírez) http://www.mpatlas.org/mpa/sites/68808466/json/
68808466 4249
4249
Archipiélago de Revillagigedo http://www.mpatlas.org/mpa/sites/68808404/json/
68808

In [161]:
mods = '''
68808485 BRA BRA !NEW! Naufrágio Queimado Parque Estadual Marinho 2018-10-12 12:20:53.423198
68807800 GBR GBR Flamborough Head Special Areas of Conservation (Habitats Directive) 2018-10-08 13:52:33.050569
68808483 GBR GBR !NEW! Flamborough Head [No-Take Zone] Special Areas of Conservation (Habitats Directive) 2018-10-08 13:56:54.576230
9450 GBR GBR North of Lundy Marine Conservation Zone 2018-10-08 14:04:09.292634
15066 GBR GBR Lundy Marine Conservation Zone 2018-10-08 14:43:41.824497
68808484 GBR GBR !NEW! Lundy [no-take zone] Marine Conservation Zone 2018-10-08 14:44:47.656833
'''

In [311]:
modids = (68808485, 68807800, 68808483, 9450, 15066, 68808484)

def comma_splitter(tag_string):
    return [t.strip() for t in tag_string.split(',') if t.strip()]

In [337]:
import taggit

mpatlas_pgweb = '/Users/russmo/Code/postgres/mpatlas_20181012_webedits.gpkg'
mds = DataSource(mpatlas_pgweb)

allfields = mds[0].fields.copy()
removefields = ['mpa_id', 'categories', ]
fields = [f for f in allfields if f not in removefields]

for feat in mds['mpatlas_20181012_webedits']:
    print(feat['mpa_id'].value, feat['name'].value, feat['designation'].value)
    try:
        m, created = Mpa.objects.get_or_create(name=feat['name'].value, designation=feat['designation'].value, verification_state=feat['verification_state'].value)
        for f in fields:
            setattr(m, f, feat[f].value)
        m.geom = feat.geom.geos
        m.save()
        m.categories.clear()
        m.categories.add(*comma_splitter(feat['categories'].value))
        print(m.pk)
    except(Mpa.DoesNotExist):
        print('  Error')

68808485 Naufrágio Queimado Parque Estadual Marinho
Carto Error for mpa_id 68808741: Access denied
68808741
68808484 Lundy [no-take zone] Marine Conservation Zone
Carto Error for mpa_id 68808742: Access denied
68808742
68808483 Flamborough Head [No-Take Zone] Special Areas of Conservation (Habitats Directive)
Carto Error for mpa_id 68808743: Access denied
68808743
68807800 Flamborough Head Special Areas of Conservation (Habitats Directive)
Carto Error for mpa_id 68807800: Access denied
68807800
15066 Lundy Marine Conservation Zone
Carto Error for mpa_id 15066: Access denied
15066
9450 North of Lundy Marine Conservation Zone
Carto Error for mpa_id 9450: Access denied
9450


In [332]:
import datetime
mm = Mpa.objects.filter(name__icontains='Lundy [no-take zone]')
list(mm.values_list('created_date', 'mpa_id', 'name', 'designation'))
mm.filter(created_date__gte=datetime.datetime(2018,10,12))

<QuerySet []>